# 080420 verbosify apostrophes and plural

In [132]:
for i in range(10):
    if i == 6: break

print(i)

6


In [139]:
[print(s) for s in [{"I'm", 'I am', 'ur boy is'}, {'wassup'}]]

{"I'm", 'I am', 'ur boy is'}
{'wassup'}


[None, None]

In [177]:
import random
import re
from nltk.corpus import wordnet
from nltk import pos_tag

# -- Helper functions for verbosify -- #
MISSPELLINGS = {'im': "I'm", "i'm": "I'm", 'Im': "I'm", 'i': 'I'}
WHITELIST = [['a', 'an', 'the'],
             ['I', 'me', 'ur boy', 'me, myself and I', 'yours truly'],
             ['you', 'thou', 'thoust'],
             ['will', 'shall', 'shalt'],
             ["I'm", 'I am', 'ur boy is'],
             ["can't", 'cannot', 'unable', 'shant'],
             ["shouldn't", 'shant', "shalln't"],
             ["you're", 'you are']]

def get_word_list(input_sentence):
    l = []
    temp = [v for v in re.split('(\W)', input_sentence) if v != '']
    
    i = 0
    while i < len(temp):
        if temp[i] in ["'", '’'] and i > 0 and i < len(temp)-1:
            # check possessive
            if temp[i+1] == 's': l.append("'s")
            # check contraction
            elif temp[i-1] != ' ' and temp[i+1] != ' ': l[-1] += "'" + temp[i+1]
            
            i += 2
        else:
            l.append(temp[i])
            i += 1
    
    return l
            

def get_synonym(word, pos):
    synsets = wordnet.synsets(word)
    synonyms = []

    # loop through all synsets
    for synset in synsets:
        # don't check synset if wrong part of speech
        if synset.name().split('.')[1] not in pos: continue

        # loop through each synonym
        for synonym in synset.lemmas():
            synonym = synonym.name()
            if synonym != word and synonym not in synonyms: synonyms.append(synonym)
    
    # no unique synonyms?
    if not synsets or not synonyms: return word
    # otherwise, choose random synonym
    return random.choice(synonyms).replace('_', ' ')

def get_whitelist_synonym(word):
    # get the correct set
    for synonyms in WHITELIST:
        if word in synonyms: break

    return random.choice(synonyms) # return random synonym

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'): return 'as'
    elif treebank_tag.startswith('V'): return 'v'
    elif treebank_tag.startswith('N'): return 'n'
    elif treebank_tag.startswith('R'): return 'r'
    else: return ''


# -- main verbosify function -- #
def verbosify(input_sentence):
    new_sentence = ''

    # go through every word
    for word, pos in pos_tag(get_word_list(input_sentence)):
        # punctuation/whitespace/possessive, whitelist, whitelist misspellings, normal word
        if re.match(r'[^\w]', word) or word == "'s": new_sentence += word
        elif any([word in s for s in WHITELIST]): new_sentence += get_whitelist_synonym(word)
        elif word in MISSPELLINGS: new_sentence += get_whitelist_synonym(MISSPELLINGS[word])
        else: new_sentence += get_synonym(word, get_wordnet_pos(pos))

    # return the sentence
    return new_sentence

In [178]:
print(verbosify("i'm cool. basically, the last two times i was downstairs for an extended period of time, i felt a sharp stinging pain where the medication was applied"))

I am aplomb. fundamentally, an lowest two metre yours truly be downstairs for a pass period of time of metre, me matte up a shrill sting bother where the medicinal drug exist use


In [43]:
pos_tag(["you're", 'sad', 'about', "the", "mechanics", 'cars'])

[("you're", 'RB'),
 ('sad', 'JJ'),
 ('about', 'IN'),
 ('the', 'DT'),
 ("mechanics'", 'NN'),
 ('cars', 'NNS')]

In [44]:
from nltk.tokenize import word_tokenize

In [122]:
get_synonym('unable', '')

'unable'

In [172]:
input_sentence = "im testing some misspellings — some “punctuation” — and I’d love to try English’s common contractions, too!"
verbosify(input_sentence)

'I am try some misspelling — some “punctuation mark” — and me’500 sexual love to judge side’southward vulgar condensation, as well!'

In [180]:
print(get_word_list(input_sentence))

['im', ' ', 'testing', ' ', 'some', ' ', 'misspellings', ' ', '—', ' ', 'some', ' ', '“', 'punctuation', '”', ' ', '—', ' ', 'and', ' ', "I'd", ' ', 'love', ' ', 'to', ' ', 'try', ' ', 'English', "'s", ' ', 'common', ' ', 'contractions', ',', ' ', 'too', '!']


In [176]:
'’' == "'"

False

# is gone

In [ ]:
from PIL import Image, ImageDraw


msg = "hello"

im = Image.new("RGBA",(W,H),"yellow")
draw = ImageDraw.Draw(im)
w, h = draw.textsize(msg)
draw.text(((W-w)/2,(H-h)/2), msg, fill="black")

im.save("hello.png", "PNG")

In [41]:
'burh is gone'[:-8]

'burh'

In [44]:
text = 'bruh is gone'

In [97]:
from PIL import Image, ImageDraw, ImageSequence, ImageFile, ImageFont
import io
ImageFile.LOAD_TRUNCATED_IMAGES = True

W, H = (352,200)
im = Image.open('../images/crabrave.gif')

frames = []
# Loop over each frame in the animated image
for frame in ImageSequence.Iterator(im):
    frame = frame.convert('RGB')
    
    # Draw the text on the frame
    d = ImageDraw.Draw(frame)
    color = '#fff'
    
    # draw message
    myFont = ImageFont.truetype("GILLSANS.ttf", 42)
    top_msg = text[:-8].upper()
    w, h = d.textsize(top_msg, font=myFont)
    d.text(((W-w)/2, 50), top_msg, font=myFont, fill=color)
    
    w, h = d.textsize('IS GONE', font=myFont)
    d.text(((W-w)/2, 100), 'IS GONE', font=myFont, fill=color)
    
    # draw line
    d.line((int(W*0.15),H/2, int(W*0.85),H/2), fill=color)
    
    del d
    
    # save
    b = io.BytesIO()
    frame.save(b, format="GIF")
    frame = Image.open(b)

    # Then append the single frame image to a list of frames
    frames.append(frame)
    
# save frames as GIF
frames[0].save('out.gif', save_all=True, append_images=frames[1:])

In [91]:
im = Image.open('../images/crabrave.gif')

for frame in ImageSequence.Iterator(im):
    frame = frame.convert('RGB')

In [82]:
im = Image.open('../images/crabrave.gif').convert('RGB')


<PIL.Image.Image image mode=RGB size=352x200 at 0x7FD71A30B190>


In [8]:
im = Image.open('../images/crabrave.gif')
ImageFile.LOAD_TRUNCATED_IMAGES = True

for frame in ImageSequence.Iterator(im):
    im.show()